### $\color{#ddd}{\text{Ejemplos de uso del FIC (FLOPs Information Criterion)}}$
### $\color{#ddd}{\text{Comparación con criterios tradicionales: AIC, BIC, y otros}}$

# $\color{#dda}{\text{0. Libs import}}$

Para la fecha en la que hago este jupyter no he hecho un paquete instalable universalmente así que se debe especificar el contexto de dónde está la librería de flop_counter.

In [2]:
import sys
import os

project_path = r'C:\Users\hecto\OneDrive\Escritorio\Personal\iroFactory\31.FLOPs-Information-Criterion'
if project_path not in sys.path:
    sys.path.insert(0, project_path)

# Fix OpenMP
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

import numpy as np
import flop_counter
from flop_counter.flop_information_criterion import FlopInformationCriterion

np.random.seed(42)

# $\color{#dda}{\text{1. Declaración de funciones para criterios}}$

Para comparación.

In [3]:
# Criterios tradicionales
def calculate_aic(log_likelihood: float, k: int) -> float:
    """
    AIC: Akaike Information Criterion
    AIC = -2*log(L) + 2*k
    """
    return log_likelihood + 2 * k


def calculate_bic(log_likelihood: float, k: int, n: int) -> float:
    """
    BIC: Bayesian Information Criterion
    BIC = -2*log(L) + k*log(n)
    """
    return log_likelihood + k * np.log(n)


def calculate_hqic(log_likelihood: float, k: int, n: int) -> float:
    """
    HQIC: Hannan-Quinn Information Criterion
    HQIC = -2*log(L) + 2*k*log(log(n))
    """
    return log_likelihood + 2 * k * np.log(np.log(n))


def calculate_mdl(log_likelihood: float, k: int, n: int) -> float:
    """
    MDL: Minimum Description Length
    MDL = -log(L) + (k/2)*log(n)
    Nota: Similar a BIC pero con diferentes constantes
    """
    return log_likelihood / 2 + (k / 2) * np.log(n)

In [4]:
# Comparaciones
def print_criteria_comparison(name: str, results: dict, show_all: bool = False):
    """Imprime comparación de criterios de información."""
    
    print(f"\n{'='*80}")
    print(f"MODELO: {name}")
    print(f"{'='*80}")
    
    # Información básica
    print(f"\nInformación del Modelo:")
    print(f"  Parámetros:       {results.get('n_params', 'N/A'):,}")
    print(f"  FLOPs:            {results.get('flops', 'N/A'):,}")
    print(f"  Muestras:         {results.get('n_samples', 'N/A')}")
    print(f"  Log-Likelihood:   {results.get('log_likelihood_term', 'N/A'):.2f}")
    
    if 'accuracy' in results:
        acc_metric = 'R²' if results.get('accuracy', 0) <= 1 else 'Accuracy'
        print(f"  {acc_metric}:            {results['accuracy']:.4f}")
    
    # Criterios de información
    print(f"\nCriterios de Información:")
    print(f"  {'Criterio':<15} {'Valor':<15} {'Penalización':<20}")
    print(f"  {'-'*50}")
    
    # AIC
    if 'aic' in results:
        print(f"  {'AIC':<15} {results['aic']:<15.2f} {'2k':<20}")
    
    # BIC
    if 'bic' in results:
        print(f"  {'BIC':<15} {results['bic']:<15.2f} {'k*log(n)':<20}")
    
    # FIC (destacado)
    print(f"  {'FIC (RIC)':<15} {results['fic']:<15.2f} {'2*log(FLOPs) + k':<20} *")
    
    # Criterios adicionales (solo si show_all=True)
    if show_all:
        if 'hqic' in results:
            print(f"  {'HQIC':<15} {results['hqic']:<15.2f} {'2k*log(log(n))':<20}")
        
        if 'mdl' in results:
            print(f"  {'MDL':<15} {results['mdl']:<15.2f} {'(k/2)*log(n)':<20}")
    
    # Desglose del FIC
    print(f"\nDesglose del FIC:")
    print(f"  Ajuste (likelihood):       {results['log_likelihood_term']:.2f}")
    print(f"  Penalización FLOPs:        {results['flops_penalty']:.2f}")
    print(f"  Penalización parámetros:   {results['params_penalty']:.2f}")
    print(f"  Coeficientes: α={results['alpha']:.2f}, β={results['beta']:.2f}")


def compare_all_criteria(models_results: dict, show_all: bool = False):
    """Compara todos los modelos según diferentes criterios."""
    
    print(f"\n{'='*80}")
    print("COMPARACIÓN DE MODELOS")
    print(f"{'='*80}")
    
    # Preparar datos
    model_names = list(models_results.keys())
    criteria = ['AIC', 'BIC', 'FIC']
    
    if show_all:
        criteria.extend(['HQIC', 'MDL'])
    
    # Tabla comparativa
    print(f"\n{'Modelo':<20}", end="")
    for criterion in criteria:
        print(f"{criterion:>12}", end="")
    print()
    print("-" * (20 + 12 * len(criteria)))
    
    for name in model_names:
        res = models_results[name]
        print(f"{name:<20}", end="")
        
        for criterion in criteria:
            key = criterion.lower()
            value = res.get(key, float('nan'))
            print(f"{value:>12.2f}", end="")
        print()
    
    # Encontrar mejores modelos según cada criterio
    print(f"\n{'Criterio':<15} {'Mejor Modelo':<20} {'Valor':<15}")
    print("-" * 50)
    
    for criterion in criteria:
        key = criterion.lower()
        best_name = min(model_names, key=lambda x: models_results[x].get(key, float('inf')))
        best_value = models_results[best_name][key]
        print(f"{criterion:<15} {best_name:<20} {best_value:<15.2f}")
    
    # Calcular diferencias respecto al mejor FIC
    print(f"\n{'='*80}")
    print("ΔFIC: Diferencias respecto al mejor modelo según FIC")
    print(f"{'='*80}")
    
    fic_values = {name: res['fic'] for name, res in models_results.items()}
    best_fic = min(fic_values.values())
    
    print(f"\n{'Modelo':<20} {'FIC':<15} {'ΔFIC':<15} {'Interpretación':<30}")
    print("-" * 80)
    
    for name in sorted(model_names, key=lambda x: fic_values[x]):
        fic = fic_values[name]
        delta_fic = fic - best_fic
        
        # Interpretación según diferencia
        if delta_fic < 2:
            interpretation = "Equivalente al mejor"
        elif delta_fic < 10:
            interpretation = "Evidencia sustancial contra"
        else:
            interpretation = "Evidencia fuerte contra"
        
        marker = "* MEJOR" if delta_fic < 0.01 else ""
        
        print(f"{name:<20} {fic:<15.2f} {delta_fic:<15.2f} {interpretation:<30} {marker}")

# $\color{#dda}{\text{2. Ejemplos}}$

### $\color{#dda}{\text{Ejemplo 1: Regresión Lineal Simple}}$

In [5]:
# Generar datos sintéticos
n_samples = 100
X_train = np.random.randn(n_samples, 1)
true_slope = 2.5
true_intercept = 1.0
noise = np.random.randn(n_samples) * 0.5
y_train = true_slope * X_train.squeeze() + true_intercept + noise

# Definir modelos de diferentes complejidades
def linear_model_simple(X):
    """Modelo lineal simple: y = ax + b"""
    W = np.array([[true_slope], [true_intercept]])
    X_extended = np.column_stack([X, np.ones(len(X))])
    return X_extended @ W

def linear_model_complex(X):
    """Modelo lineal con más operaciones innecesarias"""
    # Mismo resultado pero con más FLOPs
    W = np.array([[true_slope], [true_intercept]])
    X_extended = np.column_stack([X, np.ones(len(X))])
    
    # Operaciones adicionales innecesarias (más FLOPs)
    temp = X_extended @ W
    temp = np.exp(np.log(temp))  # identidad costosa
    temp = temp @ np.eye(1)      # multiplicación innecesaria
    
    return temp

In [6]:
# Evaluar modelos
fic_calc = FlopInformationCriterion(variant='hybrid')

result_simple = fic_calc.evaluate_model(
    model=linear_model_simple,
    X=X_train,
    y_true=y_train,
    task='regression',
    n_params=2,  # pendiente + intercept
    framework='numpy'
)

result_complex = fic_calc.evaluate_model(
    model=linear_model_complex,
    X=X_train,
    y_true=y_train,
    task='regression',
    n_params=2,  # mismo número de parámetros
    framework='numpy'
)

C:\Users\hecto\OneDrive\Escritorio\Personal\iroFactory\31.FLOPs-Information-Criterion\flop_counter\flop_information_criterion.py:234: UserWarning: FLOPs = 0, penalización por FLOPs es 0
  warnings.warn("FLOPs = 0, penalización por FLOPs es 0", UserWarning)
C:\Users\hecto\AppData\Local\Temp\ipykernel_18284\482758071.py:24: RuntimeWarning: invalid value encountered in log
  temp = np.exp(np.log(temp))  # identidad costosa


In [7]:
# Calcular criterios tradicionales para ambos modelos
for name, result in [("Simple", result_simple), ("Complejo", result_complex)]:
    result['aic'] = calculate_aic(result['log_likelihood_term'], result['n_params'])
    result['bic'] = calculate_bic(result['log_likelihood_term'], result['n_params'], result['n_samples'])
    result['hqic'] = calculate_hqic(result['log_likelihood_term'], result['n_params'], result['n_samples'])
    result['mdl'] = calculate_mdl(result['log_likelihood_term'], result['n_params'], result['n_samples'])

# Mostrar resultados
print_criteria_comparison("Modelo Simple", result_simple, show_all=True)
print_criteria_comparison("Modelo Complejo", result_complex, show_all=True)

# Comparación
models_linear = {
    'Simple': result_simple,
    'Complejo': result_complex
}
compare_all_criteria(models_linear, show_all=True)

print("\n Observación:")
print("   AIC y BIC son idénticos (mismo # de parámetros)")
print("   FIC correctamente penaliza el modelo complejo (más FLOPs)")


MODELO: Modelo Simple

Información del Modelo:
  Parámetros:       2
  FLOPs:            0
  Muestras:         100
  Log-Likelihood:   975.95
  R²:            0.0000

Criterios de Información:
  Criterio        Valor           Penalización        
  --------------------------------------------------
  AIC             979.95          2k                  
  BIC             985.16          k*log(n)            
  FIC (RIC)       977.95          2*log(FLOPs) + k     *
  HQIC            982.06          2k*log(log(n))      
  MDL             492.58          (k/2)*log(n)        

Desglose del FIC:
  Ajuste (likelihood):       975.95
  Penalización FLOPs:        0.00
  Penalización parámetros:   2.00
  Coeficientes: α=2.00, β=1.00

MODELO: Modelo Complejo

Información del Modelo:
  Parámetros:       2
  FLOPs:            0
  Muestras:         100
  Log-Likelihood:   nan
  R²:            0.0000

Criterios de Información:
  Criterio        Valor           Penalización        
  -----------------

### $\color{#dda}{\text{Ejemplo 2: Redes Neuronales de Diferentes Arquitecturas}}$

In [8]:
# Generar datos de clasificación
n_samples = 200
n_features = 20
n_classes = 3

X_train = np.random.randn(n_samples, n_features)
y_train = np.random.randint(0, n_classes, n_samples)
y_train_onehot = np.eye(n_classes)[y_train]

def softmax(x):
    """Softmax estable numéricamente"""
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

In [9]:
# Modelo 1: Red poco profunda y ancha
def wide_shallow_net(X):
    """Red ancha y poco profunda: 20 -> 100 -> 3"""
    W1 = np.random.randn(20, 100) * 0.01
    b1 = np.zeros(100)
    W2 = np.random.randn(100, 3) * 0.01
    b2 = np.zeros(3)
    
    h1 = np.maximum(0, X @ W1 + b1)  # ReLU
    logits = h1 @ W2 + b2
    return softmax(logits)

# Modelo 2: Red profunda y estrecha
def deep_narrow_net(X):
    """Red profunda y estrecha: 20 -> 30 -> 30 -> 30 -> 3"""
    W1 = np.random.randn(20, 30) * 0.01
    b1 = np.zeros(30)
    W2 = np.random.randn(30, 30) * 0.01
    b2 = np.zeros(30)
    W3 = np.random.randn(30, 30) * 0.01
    b3 = np.zeros(30)
    W4 = np.random.randn(30, 3) * 0.01
    b4 = np.zeros(3)
    
    h1 = np.maximum(0, X @ W1 + b1)
    h2 = np.maximum(0, h1 @ W2 + b2)
    h3 = np.maximum(0, h2 @ W3 + b3)
    logits = h3 @ W4 + b4
    return softmax(logits)

# Modelo 3: Red balanceada
def balanced_net(X):
    """Red balanceada: 20 -> 50 -> 25 -> 3"""
    W1 = np.random.randn(20, 50) * 0.01
    b1 = np.zeros(50)
    W2 = np.random.randn(50, 25) * 0.01
    b2 = np.zeros(25)
    W3 = np.random.randn(25, 3) * 0.01
    b3 = np.zeros(3)
    
    h1 = np.maximum(0, X @ W1 + b1)
    h2 = np.maximum(0, h1 @ W2 + b2)
    logits = h2 @ W3 + b3
    return softmax(logits)

In [10]:
# Calcular parámetros para cada modelo
n_params_wide = (20*100 + 100) + (100*3 + 3)  # 2303
n_params_deep = (20*30 + 30) + (30*30 + 30) + (30*30 + 30) + (30*3 + 3)  # 2523
n_params_balanced = (20*50 + 50) + (50*25 + 25) + (25*3 + 3)  # 2353

results_nn = {}

for name, model, n_params in [
    ("Wide-Shallow", wide_shallow_net, n_params_wide),
    ("Deep-Narrow", deep_narrow_net, n_params_deep),
    ("Balanced", balanced_net, n_params_balanced)
]:
    print(f"  Evaluando: {name}...")
    
    result = fic_calc.evaluate_model(
        model=model,
        X=X_train,
        y_true=y_train,
        task='classification',
        n_params=n_params,
        framework='numpy'
    )
    
    # Agregar criterios tradicionales
    result['aic'] = calculate_aic(result['log_likelihood_term'], n_params)
    result['bic'] = calculate_bic(result['log_likelihood_term'], n_params, n_samples)
    result['hqic'] = calculate_hqic(result['log_likelihood_term'], n_params, n_samples)
    result['mdl'] = calculate_mdl(result['log_likelihood_term'], n_params, n_samples)
    
    results_nn[name] = result
    
    print_criteria_comparison(name, result, show_all=False)

# Comparación completa
compare_all_criteria(results_nn, show_all=False)

print("\n Observación:")
print("   FIC captura el trade-off entre parámetros y FLOPs")
print("   Modelos con más parámetros no siempre son más costosos computacionalmente")

  Evaluando: Wide-Shallow...

MODELO: Wide-Shallow

Información del Modelo:
  Parámetros:       2,403
  FLOPs:            1,200
  Muestras:         200
  Log-Likelihood:   439.48
  R²:            0.3350

Criterios de Información:
  Criterio        Valor           Penalización        
  --------------------------------------------------
  AIC             5245.48         2k                  
  BIC             13171.33        k*log(n)            
  FIC (RIC)       2856.66         2*log(FLOPs) + k     *

Desglose del FIC:
  Ajuste (likelihood):       439.48
  Penalización FLOPs:        14.18
  Penalización parámetros:   2403.00
  Coeficientes: α=2.00, β=1.00
  Evaluando: Deep-Narrow...

MODELO: Deep-Narrow

Información del Modelo:
  Parámetros:       2,583
  FLOPs:            1,200
  Muestras:         200
  Log-Likelihood:   439.44
  R²:            0.3900

Criterios de Información:
  Criterio        Valor           Penalización        
  --------------------------------------------------
 

### $\color{#dda}{\text{Ejemplo 3: Comparación con PyTorch}}$

In [11]:
import torch
import torch.nn as nn

In [12]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc = nn.Linear(32 * 8 * 8, 10)
    
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return torch.softmax(x, dim=1)

class DeepCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.fc = nn.Linear(64 * 2 * 2, 10)
    
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv3(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv4(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return torch.softmax(x, dim=1)

In [13]:
# Datos dummy
X_torch = torch.randn(16, 3, 32, 32)
y_torch = torch.randint(0, 10, (16,))

# Evaluar modelos
simple_cnn = SimpleCNN()
deep_cnn = DeepCNN()

results_torch = {}

In [14]:
for name, model in [("SimpleCNN", simple_cnn), ("DeepCNN", deep_cnn)]:
    print(f"\nEvaluando: {name}...")
    
    result = fic_calc.evaluate_model(
        model=model,
        X=X_torch,
        y_true=y_torch.numpy(),
        task='classification',
        framework='torch'
    )
    
    # Agregar criterios tradicionales
    n_params = result['n_params']
    result['aic'] = calculate_aic(result['log_likelihood_term'], n_params)
    result['bic'] = calculate_bic(result['log_likelihood_term'], n_params, 16)
    
    results_torch[name] = result
    print_criteria_comparison(name, result, show_all=False)

compare_all_criteria(results_torch, show_all=False)

print("\n Observación:")
print("   FIC refleja la diferencia en profundidad de la red")
print("   DeepCNN tiene más FLOPs a pesar de entrada pequeña")


Evaluando: SimpleCNN...

MODELO: SimpleCNN

Información del Modelo:
  Parámetros:       25,578
  FLOPs:            186,122,240
  Muestras:         16
  Log-Likelihood:   69.79
  R²:            0.2500

Criterios de Información:
  Criterio        Valor           Penalización        
  --------------------------------------------------
  AIC             51225.79        2k                  
  BIC             70987.07        k*log(n)            
  FIC (RIC)       26058.12        2*log(FLOPs) + k     *

Desglose del FIC:
  Ajuste (likelihood):       69.79
  Penalización FLOPs:        410.33
  Penalización parámetros:   25578.00
  Coeficientes: α=2.00, β=1.00

Evaluando: DeepCNN...

MODELO: DeepCNN

Información del Modelo:
  Parámetros:       63,082
  FLOPs:            110,624,768
  Muestras:         16
  Log-Likelihood:   73.66
  R²:            0.1875

Criterios de Información:
  Criterio        Valor           Penalización        
  --------------------------------------------------
  AIC 

# $\color{#dda}{\text{3. Conclusión}}$

### $\color{#dda}{\text{Ventajas del FIC:}}$

$\color{#ddd}{\text{1. Captura complejidad computacional real}}$

$\color{#ddd}{\text{    - AIC/BIC solo consideran número de parámetros}}$

$\color{#ddd}{\text{    - FIC considera FLOPs (costo de ejecución)}}$
<br>
<br>
<br>

$\color{#ddd}{\text{2. Distingue modelos con igual número de parámetros}}$

$\color{#ddd}{\text{    - Dos redes con 1000 parámetros pueden tener FLOPs muy diferentes}}$

$\color{#ddd}{\text{    - FIC penaliza apropiadamente el modelo más costoso}}$
<br>
<br>
<br>

$\color{#ddd}{\text{3. Refleja la arquitectura del modelo}}$

$\color{#ddd}{\text{    - Profundidad, ancho, conexiones skip}}$

$\color{#ddd}{\text{    - Todo se traduce en FLOPs}}$
<br>
<br>
<br>

$\color{#ddd}{\text{4. Útil para deployment}}$

$\color{#ddd}{\text{    - Selecciona modelos eficientes para producción}}$

$\color{#ddd}{\text{    - Balance entre precisión y costo computacional}}$
<br>
<br>
<br>

$\color{#ddd}{\text{Cuándo usar:}}$

$\color{#ddd}{\text{    AIC:  Selección de modelos cuando muestra es pequeña}}$

$\color{#ddd}{\text{    BIC:  Cuando se prefiere modelos más simples (penaliza más)}}$

$\color{#ddd}{\text{    FIC:  Cuando el costo computacional es importante (deployment, móvil, edge)}}$
<br>
<br>
<br>

$\color{#ddd}{\text{RECOMENDACIÓN:}}$

$\color{#ddd}{\text{Use FIC-Hybrid (α=2, β=1) para balance entre:}}$

$\color{#ddd}{\text{    - Ajuste a los datos}}$

$\color{#ddd}{\text{    - Complejidad computacional (FLOPs)}}$

$\color{#ddd}{\text{    - Complejidad paramétrica (parámetros)}}$